In [1]:
# LINEAR PROGRAMMING PACKAGE
import pulp

### Problem
Maximize: <br>
$Z = 4x + 3y$

Contraints:<br>
$x \ge 0$ <br>
$y \ge 2$ <br>
$2y \le 25-x$ <br>
$4y \ge 2x-8$ <br>
$y \le 2x-5$ <br>

In [2]:
# INSTANTIATE LP OBJECT
problem = pulp.LpProblem('MaxFunction',pulp.LpMaximize)


# INSTANTIATE LP VARIABLES
x = pulp.LpVariable('x', lowBound = 0, cat = 'Continuous') # X-VALUE IS >= 0
y = pulp.LpVariable('y', lowBound = 2, cat = 'Continuous') # Y-VALUE IS >= 2

The Objective function is added first. Followed by the constraints. 

In [3]:
# OBJECTIVE FUNCTION
problem += 4*x + 3*y, 'Z'

# Constraints
problem += 2*y <= 25 - x
problem += 4*y >= 2*x - 8
problem += y <= 2*x - 5

In [4]:
print(problem)

MaxFunction:
MAXIMIZE
4*x + 3*y + 0
SUBJECT TO
_C1: x + 2 y <= 25

_C2: - 2 x + 4 y >= -8

_C3: - 2 x + y <= -5

VARIABLES
x Continuous
2 <= y Continuous



In [5]:
# SOLVE THE LP PROBLEM
problem.solve()

# DETERMINE THE STATUS OF THE LP PROBLEM
status = pulp.LpStatus[problem.status]
print('Status:',status)

Status: Optimal


In [6]:
print([variable.varValue for variable in problem.variables()])

[14.5, 5.25]


In [7]:
# DETERMINE THE OBJECTIVE FUNCTION USING THE VARIABLES
maxVal = pulp.value(problem.objective)
print(maxVal)

73.75


## Real World Example: Blending Problem

<table style="width:75%">
  <tr>
    <th>Ingredient</th>
    <th>Cost (Euro/kg) </th> 
    <th>Availability (kg) </th>
  </tr>
  <tr>
    <td>Pork</td>
    <td>4.32</td> 
    <td>30</td>
  </tr>
  <tr>
    <td>Wheat</td>
    <td>2.46</td> 
    <td>20</td>
  </tr>
  <tr>
    <td>Starch</td>
    <td>1.86</td> 
    <td>17</td>
  </tr>
</table>

There are  2 types of sausage:
- Economy (>40% Pork)
- Premium (>60% Pork)

Note: <br>
- One sausage is 50 grams (0.05 kg) <br>
- According to government regulations, the most starch we can use in our sausages is 25%
- 23kg of pork was bought and must go on the sausage.
- There is a demand for 350 economy sausages and 500 premium sausages.

Problem: Determine how to most cost effectively blend the sausages.

Variables: <br>

$p_e=$Pork in the economy sausages (kg) <br>
$w_e=$Wheat in the economy sausages (kg) <br>
$s_e=$Starch in the economy sausages (kg) <br>
$p_p=$Pork in the premium sausages (kg) <br>
$w_p=$Wheat in the premium sausages (kg) <br>
$s_p=$Starch in the premium sausages (kg) <br>

Objective Function: <br>
$Cost=4.32(p_e+p_p)+2.46(w_e+w_p)+1.86(s_e+s_p)$

Constraints: <br>
$p_e+w_e+s_e=350 \times 0.05$ <br>
$p_p+w_p+s_p=500 \times 0.05$ <br>
$p_e≥0.4(p_e+w_e+s_e)$ <br>
$p_p≥0.6(p_p+w_p+s_p)$ <br>
$s_e≤0.25(p_e+w_e+s_e)$ <br>
$s_p≤0.25(p_p+w_p+s_p)$ <br>
$p_e+p_p≤30$ <br>
$w_e+w_p≤20$ <br>
$s_e+s_p≤17$ <br>
$p_e+p_p≥23$ <br>

In [8]:
import pandas as pd

# WRITE THE TABLE DATA INTO A DICTIONARY
dictVar = {'Ingredient':['Pork','Wheat','Starch'],
          'Cost':[4.32,2.46,1.86],
          'Availability': [30, 20, 17]}

# CREATE A PANDAS DATAFRAME USING THE DICTIONARY ABOVE
df = pd.DataFrame(dictVar)

print(df)


  Ingredient  Cost  Availability
0       Pork  4.32            30
1      Wheat  2.46            20
2     Starch  1.86            17


In [9]:
# INSTANTIATE LP OBJECT
model = pulp.LpProblem('MinCostBlenderProblem',pulp.LpMinimize)


# CREATE STRING VARIABLES LIST
sausageType = ['Economy','Premium']


# CREATE THE VARIABLES: p_e , p_s , s_e, s_p , w_p , w_e
# THE LP DICTIONARY STORED THE VARIABLES IN KEYS ('sausage_type','ingredient')
varDict = pulp.LpVariable.dicts("weight kg",
                                     ((i, j) for i in sausageType for j in df['Ingredient']),
                                     lowBound=0,
                                     cat='Continuous')

In [10]:
# OBJECTIVE FUNCTION

# LIST WILL STORE THE THREE SUMS VARIABLES
obj = [0 ,0 ,0]

# ITERATE EACH ROW OF THE DATAFRAME
for ii,row in df.iterrows():
    obj[ii] = row['Cost'] * ( varDict['Economy',row['Ingredient']] + varDict['Premium',row['Ingredient']] )

# 'lpSum' WILL ADD ALL THE STATEMENTS TOGETHER TO CREATE ONE EQUATION
model += pulp.lpSum(obj)
    

In [11]:
# CONSTRAINTS


# 350 ECONOMY AND 500 PREMIUM AT 0.05 KG
const1 = [varDict['Economy',ingred] for ingred in df['Ingredient']]
model += pulp.lpSum(const1) == 350 * 0.05

const2 = [varDict['Premium',ingred] for ingred in df['Ingredient']]
model += pulp.lpSum(const2) == 500 * 0.05


# ECONOMY HAS >= 40% PORK, PREMIUM >= 60%
const3 = [varDict['Economy',ingred] for ingred in df['Ingredient']]
model += varDict['Economy','Pork'] >= 0.4 * pulp.lpSum(const3)

const4 = [varDict['Premium',ingred] for ingred in df['Ingredient']]
model += varDict['Premium','Pork'] >= 0.6 * pulp.lpSum(const4)

# SAUSAGE MUST BE <= 25%
const5 = [varDict['Economy',ingred] for ingred in df['Ingredient']]
model += varDict['Economy','Starch'] <= 0.25 * pulp.lpSum(const5)

const6 = [varDict['Premium',ingred] for ingred in df['Ingredient']]
model += varDict['Premium','Starch'] <= 0.25 * pulp.lpSum(const6)


# THERE ARE RESTRICTIONS TO HOW MUCH PORK, WEHAT, AND STARCH IS AVAILABLE

for ingred in df['Ingredient']:
    model += varDict['Economy',ingred] + varDict['Premium',ingred] <= df[ df['Ingredient'] == ingred ]['Availability']



# THERE IS 23 KG THAT MUST BE USED GO
model += varDict['Economy','Pork'] + varDict['Premium','Pork'] >= 23



#print(model)

In [12]:
# SOLVE THE LP PROBLEM
model.solve()

# DETERMINE THE STATUS OF THE LP PROBLEM
status = pulp.LpStatus[model.status]
print('Status:',status)

Status: Optimal


In [13]:
# LOOK AT EVERY LP VARIABLE
# AT NOTE WHAT THE OPTIMAL VALUE IS TO MINIMIZE THE OBJ. FUNCTION
for var in varDict:
    var_value = varDict[var].varValue
    print("The weight of {0} in {1} sausages is {2} kg".format(var[1], var[0], var_value))

The weight of Pork in Economy sausages is 8.0 kg
The weight of Wheat in Economy sausages is 5.125 kg
The weight of Starch in Economy sausages is 4.375 kg
The weight of Pork in Premium sausages is 15.0 kg
The weight of Wheat in Premium sausages is 3.75 kg
The weight of Starch in Premium sausages is 6.25 kg


In [14]:
# DETERMINE THE MINIMUL COST BASED ON THE VARIABLES ABOVE
total_cost = pulp.value(model.objective)

print("The total cost is €{} for 350 economy sausages and 500 premium sausages".format(round(total_cost, 2)))

The total cost is €140.96 for 350 economy sausages and 500 premium sausages
